# __Auto-ML을 사용하여 예측 모델 만들기__

## __0. 데이터 준비__

In [ ]:
%load_ext thanosql
%thanosql API_TOKEN={발급받은 개인 토큰}

In [2]:
%%thanosql
COPY bike_sharing_train FROM "tutorial_data/bike_sharing_data/bike_sharing_train.csv"

Success


In [3]:
%%thanosql
COPY bike_sharing_test FROM "tutorial_data/bike_sharing_data/bike_sharing_test.csv"

Success
CPU times: user 38.7 ms, sys: 15.5 ms, total: 54.2 ms
Wall time: 543 ms


## __1. 데이터 세트 확인__

In [4]:
%%thanosql
SELECT * 
FROM bike_sharing_train 
LIMIT 5

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0,1


## __2. 회귀 모델 생성__

이전 단계에서 확인한 bike_sharing 데이터 세트를 사용하여 자전거 수요 예측 회귀 모델을 만듭니다. 아래의 쿼리 구문을 실행하여 bike_regression이라는 이름의 모델을 만듭니다.

In [5]:
%%thanosql
BUILD MODEL tutorial_automl_regressor
USING AutomlRegressor
OPTIONS (
    target='count', 
    impute_type='simple', 
    datetime_attribs=['datetime'],
    time_left_for_this_task = 300
    ) 
AS
SELECT *
FROM bike_sharing_train

Success


## __3. 생성된 모델 평가__

아래의 쿼리문을 실행하여 이전 단계에서 만든 예측 모델의 성능을 평가합니다.

In [6]:
%%thanosql
EVALUATE USING tutorial_automl_regressor 
OPTIONS (
    target='count'
    ) 
AS
SELECT *
FROM bike_sharing_train

,metrics,value
0,MAE,70.1031
1,MSE,9679.4456
2,R2,0.3523
3,RMSLE,1.1241
4,MAPE,0.4526


## __4. 생성된 모델을 사용하여 자전거 대여 수량 예측__

이전 단계에서 만든 수요 예측 모델로 테스트용 데이터 세트(학습에 이용되지 않은 데이터 테이블, bike_shaing_test)에 있는 10개의 데이터에 대한 자전거 대여 수량을 예측해 봅니다.

In [7]:
%%thanosql
PREDICT USING tutorial_automl_regressor 
AS
SELECT *
FROM bike_sharing_test
LIMIT 10

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,predicted
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027,133.351361
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000,83.304852
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000,83.304852
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014,89.718215
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014,89.718215
5,2011-01-20 05:00:00,1,0,1,1,9.84,11.365,60,15.0013,104.283055
6,2011-01-20 06:00:00,1,0,1,1,9.02,10.605,60,15.0013,100.676295
7,2011-01-20 07:00:00,1,0,1,1,9.02,10.605,55,15.0013,91.813349
8,2011-01-20 08:00:00,1,0,1,1,9.02,10.605,55,19.0012,99.065202
9,2011-01-20 09:00:00,1,0,1,2,9.84,11.365,52,15.0013,108.508070
